<center><h1 class="list-group-item list-group-item-success">Importing Libraries</h1></center>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import librosa
import os
import pandas as pd
pd.set_option('display.max_columns',None)
import numpy as np
import pathlib
import csv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle

In [4]:
base_dir='/content/drive/MyDrive/ASE/Audio_Project_2/input'

In [7]:
targets=os.listdir(base_dir) 
print(targets)
# print(os.listdir(base_dir+"/"+targets[0]))
# print(base_dir+"/"+targets[0])


['UnAuthorised', 'Authorised']


In [8]:
header = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff zero_crossing_rate']
for i in range(1, 22):
    header.append('mfcc'+str(i))
header += ['label']
print(header)

['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20', 'mfcc21', 'label']


<center><h1 class="list-group-item list-group-item-success">Converting Audio data into Numerical data</h1></center>

In [10]:
os.remove('/content/drive/MyDrive/ASE/Audio_Project_2/file_data/NumericalAudioData.csv')
file = open('/content/drive/MyDrive/ASE/Audio_Project_2/file_data/NumericalAudioData.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

In [11]:
for target in targets:
    target_path=os.path.join(base_dir,target)
    for filename in os.listdir(target_path):
        data_points=[]
        data_points.append(filename)
        songname = os.path.join(target_path,filename)
        y, sr = librosa.load(songname, mono=True, duration=5)
        data_points.append(np.mean(librosa.feature.chroma_stft(y=y, sr=sr)))
        data_points.append(np.mean(librosa.feature.rms(y=y)))
        data_points.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)))
        data_points.append(np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)))
        data_points.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)))
        data_points.append(np.mean(librosa.feature.zero_crossing_rate(y)))
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        for e in mfcc:
            data_points.append(np.mean(e))
        data_points.append(target)
        file = open('/content/drive/MyDrive/ASE/Audio_Project_2/file_data/NumericalAudioData.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(data_points)

<center><h1 class="list-group-item list-group-item-success">Data Loading</h1></center>

In [12]:
df=pd.read_csv('/content/drive/MyDrive/ASE/Audio_Project_2/file_data/NumericalAudioData.csv')

***Preliminary Analysis***

In [13]:
df.shape

(337, 28)

In [14]:
df.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21,label
0,McArdle Rd 281.m4a,0.387598,0.088108,1365.242210,1771.311506,2486.902476,0.053334,-340.45258,116.455560,14.636538,25.407400,17.018755,8.232633,-4.801199,2.930575,-5.082539,6.246041,-17.434248,6.234110,-7.696851,-4.960170,0.053654,2.293727,-10.796652,-4.409066,-0.262439,-11.225334,UnAuthorised
1,McArdle Rd 294.m4a,0.379649,0.115266,1792.264666,2024.918748,3392.235043,0.080524,-289.19363,95.418740,4.794367,27.628004,17.621325,13.577344,-6.689746,2.854115,-9.384562,7.180109,-20.863210,8.464931,-8.722604,-2.975815,1.209049,-1.180737,-7.707803,-4.546563,-0.581724,-10.983594,UnAuthorised
2,McArdle Rd 296.m4a,0.412892,0.102540,1613.401037,1849.329343,2807.846867,0.077013,-309.45322,97.568780,10.731281,24.211290,19.409098,11.959641,-6.238619,3.132299,-5.832726,6.538689,-14.153038,3.603457,-7.764763,-2.176051,0.932781,-1.516711,-6.931557,-5.239154,-1.706985,-9.226336,UnAuthorised
3,McArdle Rd 276.m4a,0.427142,0.099531,2035.183097,2121.020272,3612.151758,0.099945,-297.16693,83.997170,11.963964,20.543760,14.373364,3.888435,-4.848911,4.531799,-7.568991,4.959585,-11.222337,1.552424,-8.485812,0.161838,-1.787608,-0.145852,-9.387383,-1.590610,-4.012473,-6.518033,UnAuthorised
4,McArdle Rd 273.m4a,0.415220,0.101017,2104.449409,2149.443202,3747.460938,0.094176,-295.22970,76.808395,18.061266,28.493190,13.102561,6.046939,-1.622509,3.297241,-5.781744,2.503388,-10.571022,4.849361,-9.339779,-0.834217,-1.975727,-1.275641,-8.259794,1.177253,-4.443179,-4.736060,UnAuthorised


In [15]:
df=df.drop(labels='filename',axis=1)
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21,label
0,0.387598,0.088108,1365.242210,1771.311506,2486.902476,0.053334,-340.45258,116.455560,14.636538,25.407400,17.018755,8.232633,-4.801199,2.930575,-5.082539,6.246041,-17.434248,6.234110,-7.696851,-4.960170,0.053654,2.293727,-10.796652,-4.409066,-0.262439,-11.225334,UnAuthorised
1,0.379649,0.115266,1792.264666,2024.918748,3392.235043,0.080524,-289.19363,95.418740,4.794367,27.628004,17.621325,13.577344,-6.689746,2.854115,-9.384562,7.180109,-20.863210,8.464931,-8.722604,-2.975815,1.209049,-1.180737,-7.707803,-4.546563,-0.581724,-10.983594,UnAuthorised
2,0.412892,0.102540,1613.401037,1849.329343,2807.846867,0.077013,-309.45322,97.568780,10.731281,24.211290,19.409098,11.959641,-6.238619,3.132299,-5.832726,6.538689,-14.153038,3.603457,-7.764763,-2.176051,0.932781,-1.516711,-6.931557,-5.239154,-1.706985,-9.226336,UnAuthorised
3,0.427142,0.099531,2035.183097,2121.020272,3612.151758,0.099945,-297.16693,83.997170,11.963964,20.543760,14.373364,3.888435,-4.848911,4.531799,-7.568991,4.959585,-11.222337,1.552424,-8.485812,0.161838,-1.787608,-0.145852,-9.387383,-1.590610,-4.012473,-6.518033,UnAuthorised
4,0.415220,0.101017,2104.449409,2149.443202,3747.460938,0.094176,-295.22970,76.808395,18.061266,28.493190,13.102561,6.046939,-1.622509,3.297241,-5.781744,2.503388,-10.571022,4.849361,-9.339779,-0.834217,-1.975727,-1.275641,-8.259794,1.177253,-4.443179,-4.736060,UnAuthorised


In [16]:
df.isnull().sum()

chroma_stft                   0
rmse                          0
spectral_centroid             0
spectral_bandwidth            0
rolloff zero_crossing_rate    0
mfcc1                         0
mfcc2                         0
mfcc3                         0
mfcc4                         0
mfcc5                         0
mfcc6                         0
mfcc7                         0
mfcc8                         0
mfcc9                         0
mfcc10                        0
mfcc11                        0
mfcc12                        0
mfcc13                        0
mfcc14                        0
mfcc15                        0
mfcc16                        0
mfcc17                        0
mfcc18                        0
mfcc19                        0
mfcc20                        0
mfcc21                        0
label                         0
dtype: int64

In [17]:
class_dict={}
for idx,label in enumerate(df['label'].unique().tolist()):
    class_dict[label]=idx
print(class_dict)

{'UnAuthorised': 0, 'Authorised': 1}


In [18]:
df['label']=df['label'].map(class_dict)
df=df.sample(frac=1).reset_index(drop=True)
df.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21,label
0,0.367031,0.081240,2001.713616,2142.096585,3864.312744,0.086117,-349.65817,85.649086,5.478385,-19.449510,-12.719901,1.839045,-11.279694,5.225027,-6.416802,6.644322,-4.313813,-1.271969,0.062576,0.615152,-6.385010,-3.697910,-2.995087,-3.860961,-0.373320,1.110234,1
1,0.351816,0.054407,2013.308372,1950.827458,3741.757780,0.100553,-359.49246,92.311890,3.048618,7.600127,-12.921133,-10.549616,-1.568728,9.905917,-7.663112,11.209610,-2.047396,-4.310838,-7.235730,4.463638,-8.522812,-5.745754,0.891153,-4.294840,-1.823139,-0.771620,1
2,0.417440,0.108311,2073.069593,2123.521979,3393.032367,0.102309,-288.30270,87.553550,7.217164,11.642424,29.822200,-1.863651,-15.542105,12.160411,-13.305327,4.824744,-12.695470,2.639625,-5.807128,-0.563086,-5.991003,1.404290,-13.775097,0.642064,-6.261918,-5.643380,0
3,0.422040,0.068454,1399.628367,1395.979390,2511.205487,0.083519,-337.67236,123.708210,-8.352958,0.772860,-1.911475,-8.976076,-6.099641,2.415822,-4.096744,9.791706,-2.439756,-12.397418,4.247353,1.815955,-6.831268,-5.256898,0.295395,-6.371269,-0.324907,4.873642,1
4,0.439646,0.090847,1629.712599,1956.267186,3348.033980,0.070223,-312.26746,102.299140,12.479951,18.682648,-18.953283,-0.749540,0.046536,2.729039,-7.079327,-2.185723,-15.311001,-0.049122,-6.549664,0.528436,0.359235,-10.031366,-4.782373,-4.871750,-5.451432,-4.608060,0


In [19]:
X=df.drop(labels='label',axis=1)
y=df[['label']]

In [20]:
X.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
0,0.367031,0.081240,2001.713616,2142.096585,3864.312744,0.086117,-349.65817,85.649086,5.478385,-19.449510,-12.719901,1.839045,-11.279694,5.225027,-6.416802,6.644322,-4.313813,-1.271969,0.062576,0.615152,-6.385010,-3.697910,-2.995087,-3.860961,-0.373320,1.110234
1,0.351816,0.054407,2013.308372,1950.827458,3741.757780,0.100553,-359.49246,92.311890,3.048618,7.600127,-12.921133,-10.549616,-1.568728,9.905917,-7.663112,11.209610,-2.047396,-4.310838,-7.235730,4.463638,-8.522812,-5.745754,0.891153,-4.294840,-1.823139,-0.771620
2,0.417440,0.108311,2073.069593,2123.521979,3393.032367,0.102309,-288.30270,87.553550,7.217164,11.642424,29.822200,-1.863651,-15.542105,12.160411,-13.305327,4.824744,-12.695470,2.639625,-5.807128,-0.563086,-5.991003,1.404290,-13.775097,0.642064,-6.261918,-5.643380
3,0.422040,0.068454,1399.628367,1395.979390,2511.205487,0.083519,-337.67236,123.708210,-8.352958,0.772860,-1.911475,-8.976076,-6.099641,2.415822,-4.096744,9.791706,-2.439756,-12.397418,4.247353,1.815955,-6.831268,-5.256898,0.295395,-6.371269,-0.324907,4.873642
4,0.439646,0.090847,1629.712599,1956.267186,3348.033980,0.070223,-312.26746,102.299140,12.479951,18.682648,-18.953283,-0.749540,0.046536,2.729039,-7.079327,-2.185723,-15.311001,-0.049122,-6.549664,0.528436,0.359235,-10.031366,-4.782373,-4.871750,-5.451432,-4.608060


In [21]:
y.head()

,label
0,1
1,1
2,0
3,1
4,0


In [22]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler=scaler.fit(X.values,y.values.ravel())

In [23]:
X_scaled=pd.DataFrame(data=scaler.transform(X.values),columns=X.columns)
X_scaled.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
0,-0.588830,0.140933,0.838598,1.102075,0.917838,0.175254,-0.476983,-0.549456,0.155512,-2.046196,-0.814309,0.226989,-0.838347,0.541001,-0.055766,0.517192,0.817245,-0.060469,0.909110,0.582917,-0.472878,0.011352,0.213144,0.031893,0.454824,0.114624
1,-0.912968,-1.282567,0.882566,0.192317,0.690575,0.899868,-0.855639,0.019533,-0.052669,-0.189788,-0.829944,-1.226654,0.803890,1.253931,-0.265348,1.806879,1.240792,-0.720921,-1.069105,1.289160,-0.866276,-0.570285,1.174176,-0.089792,0.091571,-0.196116
2,0.485068,1.577076,1.109188,1.013726,0.043906,0.988011,1.885421,-0.386819,0.304488,0.087634,2.491071,-0.207472,-1.559169,1.597305,-1.214156,0.003164,-0.749117,0.789657,-0.681880,0.366696,-0.400373,1.460500,-2.452657,1.294805,-1.020567,-1.000563
3,0.583066,-0.537404,-1.444580,-2.446777,-1.591336,0.044865,-0.015487,2.700713,-1.029545,-0.658341,0.025470,-1.042020,0.037660,0.113142,0.334380,1.406323,1.167468,-2.478416,2.043398,0.803279,-0.554998,-0.431438,1.026851,-0.672144,0.466954,0.736055
4,0.958154,0.650581,-0.572074,0.218191,-0.039538,-0.622493,0.962692,0.872424,0.755399,0.570802,-1.298622,-0.076746,1.077050,0.160847,-0.167178,-1.977282,-1.237906,0.205298,-0.883146,0.567004,0.768197,-1.787502,-0.228836,-0.251591,-0.817499,-0.829606


In [24]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X_scaled.values, y.values.ravel())

smote_df=pd.DataFrame(data=X_smote,columns=X.columns)
smote_df['class']=y_smote

In [25]:
smote_df['class'].value_counts()

1    187
0    187
Name: class, dtype: int64

In [26]:
X=smote_df.drop(labels='class',axis=1)
y=smote_df[['class']]

In [27]:
with open(file='/content/drive/MyDrive/ASE/Audio_Project_2/models/Scaler.pkl',mode='wb') as file:
    pickle.dump(obj=scaler,file=file)

In [28]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True,stratify=y)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(299, 26) (75, 26) (299, 1) (75, 1)


In [29]:
X_train.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
197,-1.449894,1.700759,0.231830,0.356590,0.603051,-0.585164,0.709871,-0.210382,-0.689462,0.311622,-0.894367,-0.628931,0.442404,-2.688938,0.187068,0.407427,-0.192246,0.514565,-0.664419,2.124879,-0.413276,-2.186299,1.207002,-2.083045,-1.058400,-0.495107
300,-1.552770,0.367961,-0.604055,-0.821058,-1.178491,-0.346411,0.650920,-1.032177,-1.875719,-0.290735,-0.381898,-0.322497,-1.331890,-1.555182,0.234537,0.696678,-1.243046,1.266888,-1.398701,0.970545,-1.360347,-2.827067,1.287470,-2.411237,0.138701,0.078322
325,0.965586,-0.375507,-0.613423,0.282677,-0.555344,-0.666274,-0.353780,1.522327,0.672128,0.568428,1.520214,1.210922,-0.513884,0.944030,0.421460,0.015675,-0.319286,1.384791,0.236368,-0.239166,0.665197,1.063334,-2.193728,1.832499,-0.700110,-0.721187
341,0.476341,0.240495,-0.144542,0.830966,0.683639,-0.951087,0.279694,0.230220,0.271695,0.943190,0.773268,1.490549,-1.442381,-0.214002,-0.315741,-0.698700,-1.014379,-1.036918,1.213495,-1.789283,0.755328,-0.067696,-0.779513,1.268824,-0.645840,-0.683700
212,0.685036,-1.742477,0.635271,0.761962,0.040548,0.616814,-1.702316,-0.259460,1.092214,-1.477821,-0.210522,-1.337270,0.789745,0.774277,0.456595,1.843636,0.386425,-0.025386,0.799931,0.571565,-0.795688,1.362373,0.568708,1.078809,1.482001,1.680475


In [30]:
y_train.head()

,class
197,1
300,1
325,0
341,0
212,1


In [31]:
X_test.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,mfcc21
347,0.292580,1.006771,0.639680,0.870380,0.333529,0.547580,1.424737,-0.469018,0.599494,1.051444,1.554368,0.946245,-0.324664,0.896992,-0.595443,0.696340,-0.597609,0.189015,-0.432135,-0.045547,0.474931,1.111390,-1.477736,0.392556,-0.303961,-1.091498
193,-0.567522,-0.031771,-0.915117,-0.110564,-0.698753,-1.002334,0.136702,0.679868,-0.359780,0.436967,0.920880,0.575969,-1.420867,-1.626563,-0.945259,-1.048162,-1.063988,-0.067381,-0.613151,-1.418880,0.761491,-0.224713,-1.453242,0.261351,-0.641659,-0.657530
234,-1.045081,1.520539,-0.051639,-0.047403,-0.129024,-0.267703,0.971334,-0.974089,-1.093423,0.240662,-0.576203,-0.397977,0.460504,-2.187252,1.199900,-0.053375,-0.388768,0.592454,-1.175231,1.090397,-0.854230,-1.707942,0.917124,-1.584468,-0.002463,-0.021123
335,0.304015,-1.341689,-0.380320,-0.005764,-0.385950,-0.401481,-1.378147,-1.451009,0.074955,0.432588,0.134573,0.449169,0.409716,-1.738176,0.956813,0.739580,-0.379407,0.814828,0.147352,1.476400,-0.637670,-0.818983,1.551767,-0.537524,0.243766,0.026260
241,0.451422,0.341175,-0.623974,-0.652582,-0.542794,-0.766282,0.339822,0.281907,-1.474910,0.880778,0.129166,-0.396168,2.175030,-0.111192,-2.217270,-0.851385,-0.261834,-0.273050,-0.156616,-1.094812,1.134920,1.230511,-0.507435,0.968915,-1.318431,-0.652767


In [32]:
y_test.head()

,class
347,0
193,0
234,1
335,1
241,0


In [33]:
X_train.to_csv("/content/drive/MyDrive/ASE/Audio_Project_2/splitted_data/X_train.csv",index=False)
X_test.to_csv("/content/drive/MyDrive/ASE/Audio_Project_2/splitted_data/X_test.csv",index=False)
y_train.to_csv("/content/drive/MyDrive/ASE/Audio_Project_2/splitted_data/y_train.csv",index=False)
y_test.to_csv("/content/drive/MyDrive/ASE/Audio_Project_2/splitted_data/y_test.csv",index=False)

## Thank You